# Data Analytics Assignment

## Problem Statement

We are sharing data for our live demo classes right from parents showing interest in their conversion (Parent bought the live classes pack).

## Following are the details of the data:
1. Interested Users – Users (child) who have shown interest in joining the demo classes.
2. Booked Users – Users who booked a trail class on our portal. There are cases where duplicate trails are possible, count both.
3. Trainers – Trainers assigned to Trail Batch.
4. Subscription Bought – Details of Sales that are made. Use only those cases where sale value is greater than equal to 499/-.

## Using these data, create following metrics:
1. Sales conversion with respect to Trainers
2. Sales conversion with respect to Time slot
3. Sales conversion with respect to Topic
4. Sales conversion with respect to Source (Field available in Interested Users)
5. Sales conversion wrt grade
6. Time taken for Sales conversion
7. Time slot – attendance %

Sales conversion refers to the number of users who have bought the session after showing interest and attending the trial session.

In [38]:
# importing libraries

import pandas as pd

import re

In [58]:
interested_users = pd.read_excel('assignment_data.xlsx', sheet_name = 'Interested Users')
booked_users = pd.read_excel('assignment_data.xlsx', sheet_name = 'Booked Users')
trainers = pd.read_excel('assignment_data.xlsx', sheet_name = 'Trainers')
subscription_bought = pd.read_excel('assignment_data.xlsx', sheet_name = 'Subscription Bought')

## Sales conversion with respect to Trainers

### Tidying trainer data

In [59]:
trainers.head(5)

,Trainer Name,Date,Timing,Grade,Batch size,Batch count,Batch,Topics
0,FAC_5001,16 April Thursday,11.15 am - 12 noon,"Grade 1,2",group,9.0,trial I,Rocking with rhymes
1,FAC_5002,16 April Thursday,11.15 am - 12 noon,"Grade 3,4",group,9.0,trial I,In my mind's eye
2,FAC_5003,16 April Thursday,11.15 am - 12 noon,"Grade 1,2",group,9.0,trial II,Word Whiz
3,FAC_5006,16 April Thursday,11.15 am - 12 noon,"Grade 3,4",group,7.0,trial II,Rocking with rhymes
4,FAC_5007,16 April Thursday,4:00-4:45 pm,"Grade 3,4",group,9.0,trial I,In my head or real


In [79]:
trainers.Timing.unique()

array(['11.15 am - 12 noon', '4:00-4:45 pm', '6:15 - 7:00 pm',
       '11.45 am -12.30 pm', '6.00 pm - 6.45 pm ', '4.00-4.45 pm'],
      dtype=object)

In [96]:
trainers['Timing'].apply(lambda x: re.findall('\d{1,2}(?:.|:)\d{2} ?(?:am|pm)?', x)[0])

0      11.15 am
1      11.15 am
2      11.15 am
3      11.15 am
4          4:00
         ...   
169     6.00 pm
170     6.00 pm
171     6.00 pm
172     6.00 pm
173     6.00 pm
Name: Timing, Length: 174, dtype: object

### Merging Trainer data with Booked Users

In [98]:
booked_users.Batch[0]

'fREADom Trial Class - Grades 1, 2 [I] 11:15am - Thursday, April 16, 2020'

In [99]:
booked_users['Batch'].apply(lambda x: re.findall('\d{2}:\d{2}(?:am|pm)',x)[0])

0       11:15am
1       11:15am
2       11:15am
3       11:15am
4       11:15am
         ...   
1706    06:00pm
1707    06:00pm
1708    06:00pm
1709    06:00pm
1710    06:00pm
Name: Batch, Length: 1711, dtype: object

In [10]:
subscription_bought.head()

,Child Name,Registered Number,Registered Email ID,Payment date,Sale by,Grade,Pricing,Duration,Sessions preferred,Live session details
0,CH_31846,9.111002e+11,s2m_test_21890@tests2m.com,16-03-2020 12:00:00 AM,SP_1001,2,199,0.5,NaN,NaN
1,CH_31847,9.111002e+11,NaN,16-03-2020 12:00:00 AM,SP_1002,4,199,0.5,NaN,NaN
2,CH_31848,9.111002e+11,s2m_test_21891@tests2m.com,16-03-2020 12:00:00 AM,NaN,4,199,0.5,NaN,NaN
3,CH_31849,9.111002e+11,s2m_test_21440@tests2m.com,16-03-2020 12:00:00 AM,SP_1004,3,199,0.5,NaN,NaN
4,CH_31850,9.111002e+11,s2m_test_21892@tests2m.com,17-03-2020 12:00:00 AM,NaN,5,199,1,NaN,NaN
